In [1]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
from shapely.geometry import Polygon, MultiPolygon, MultiPoint
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle

WORKING_DIRECTORY = '/media/dean/datastore/datasets/darknet_evaluate_pretrained_coco'
COCO_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data', 'coco')
COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'instances_val2014.json')
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY, 'images','val2014')
WEIGHTS_DIRECTORY = os.path.join(WORKING_DIRECTORY,'trained_weights')
LABELS_DIRECTORY = os.path.join(COCO_DIRECTORY, 'labels','val2014')
CATEGORY_NAMES = os.path.join(WORKING_DIRECTORY, 'data', 'coco.names')

In [2]:
pretrained_coco = COCO(COCO_ANNOTATIONS_FILE)
ann_ids = pretrained_coco.getAnnIds()
print(len(ann_ids))

loading annotations into memory...
Done (t=10.47s)
creating index...
index created!
291875


In [3]:
# For Run in Training Runs    
training_runs = os.listdir(os.path.join(WEIGHTS_DIRECTORY,'training_runs'))
all_runs = []
for run in training_runs:
    print('Evaluating mAP on Training Run:', os.path.join(WEIGHTS_DIRECTORY,'training_runs',run))
    # Grab iterations and sort checkpoints
    d = {}
    weights_files = glob.glob(os.path.join(WEIGHTS_DIRECTORY,'training_runs',run, '*.weights'))
    
    for fpath in weights_files:
        fname = os.path.split(fpath)[-1]
        iterations = fname[15:].split('.weights')[0]
        d[fname] = int(iterations)
    sorted_weights = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]))
    bdd_weights = OrderedDict()

    for weights_file in sorted_weights.keys():
        d = {}
        d['iterations'] = sorted_weights[weights_file]
        d['map_results_file'] = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file+'.txt')
        map_results_file = d['map_results_file']

        if not os.path.exists(d['map_results_file']+'.backup'):
            # Create annotations if labels directory not found
            if not os.path.exists(os.path.join(COCO_DIRECTORY,'labels/val2014/image_list.txt')):
                !java -jar $WORKING_DIRECTORY/data/cocotoyolo.jar "{COCO_ANNOTATIONS_FILE}" "{IMAGES_DIRECTORY}/" "all" "{COCO_DIRECTORY}/labels/val2014"
                yolo_convert_output = os.path.join(COCO_DIRECTORY, 'labels','convert2yolo_results.txt')
                #!python3 $WORKING_DIRECTORY/convert2Yolo/example.py --datasets COCO --img_path "{IMAGES_DIRECTORY}" --label "{COCO_ANNOTATIONS_FILE}" --convert_output_path "{LABELS_DIRECTORY}" --img_type ".jpg" --manipast_path $LABELS_DIRECTORY --cls_list_file $CATEGORY_NAMES | tee -a $yolo_convert_output
                
            CURRENT_WEIGHT = os.path.join(WEIGHTS_DIRECTORY, 'training_runs',run, weights_file)
            outfile = !cd $WORKING_DIRECTORY && ./darknet detector map cfg/coco.data cfg/yolov3_pretrained.cfg $CURRENT_WEIGHT | tee -a $map_results_file
            #!cd $WORKING_DIRECTORY && ./darknet detector valid cfg/coco.data cfg/yolov3_pretrained.cfg $CURRENT_WEIGHT -gpus 0 | tee -a $map_results_file
            with open(d['map_results_file']+'.backup',"w+") as f:
                outfile = '\n'.join(outfile)
                f.write(outfile)

        #Get mAP results from file
        class_stats = []
        map_stats = {}
        with open(d['map_results_file']+'.backup') as openfile:
            for line in openfile:
                for part in line.split():
                    if 'class_id' in part:
                        class_map = {}
                        id_tokens = line.split('class_id =')[1].split(',')
                        class_map['class_id'] = int(id_tokens[0].strip())

                        name_tokens = line.split('name =')[1].split(',')
                        class_map['class_name'] = name_tokens[0]

                        if 'ap =' in line:
                            ap_tokens = line.split('ap =')[1].split(',')
                            class_map['class_ap'] = ap_tokens[0].split('%')[0].strip()

                        class_stats.append(class_map)
                    elif '(mAP)' in part:
                        tokens = line.split('(mAP) =')[1]
                        map_stats['mean_avg_precision'] = tokens.split()[2].strip(',')
                    elif 'Detection' in part:
                        tokens = line.split(':')[1]
                        total_detection_time = str(''.join(tokens)).strip()
                        map_stats['total_detection_time'] = total_detection_time
        map_stats['class_stats'] = class_stats
        print(map_stats,'\n\n') 
        bdd_weights[weights_file] = map_stats

    # Make sure Dictionary is made properly
    all_runs.append((run, bdd_weights))
print(all_runs)
pickle_file = os.path.join(COCO_DIRECTORY, 'mAP_all_runs.pickle')
pickle_dict = {'all_runs': all_runs}
with open(pickle_file,"wb") as pickle_out:
    pickle.dump(pickle_dict, pickle_out)   

Evaluating mAP on Training Run: /media/dean/datastore/datasets/darknet_evaluate_pretrained_coco/trained_weights/training_runs/run1_pretrained
{'total_detection_time': '2031.000000 Seconds', 'mean_avg_precision': '65.64', 'class_stats': [{'class_id': 0, 'class_name': ' person', 'class_ap': '73.46'}, {'class_id': 1, 'class_name': ' bicycle', 'class_ap': '61.62'}, {'class_id': 2, 'class_name': ' car', 'class_ap': '66.13'}, {'class_id': 3, 'class_name': ' motorcycle', 'class_ap': '73.42'}, {'class_id': 4, 'class_name': ' airplane', 'class_ap': '79.32'}, {'class_id': 5, 'class_name': ' bus', 'class_ap': '85.18'}, {'class_id': 6, 'class_name': ' train', 'class_ap': '88.78'}, {'class_id': 7, 'class_name': ' truck', 'class_ap': '63.82'}, {'class_id': 8, 'class_name': ' boat', 'class_ap': '52.16'}, {'class_id': 9, 'class_name': ' traffic light', 'class_ap': '57.99'}, {'class_id': 10, 'class_name': ' fire hydrant', 'class_ap': '85.61'}, {'class_id': 11, 'class_name': ' stop sign', 'class_ap': '8

In [4]:
print(len(bdd_weights))

1
